In [32]:
import pandas as pd
read_file = pd.read_csv('text.txt', sep = '?', header=None)
read_file.to_csv('text.csv', index=None)
read_file

,0,1
0,Which number makes this equation true,3 X ___ = 30
1,"Which number is a common factor of 72, 63, and 54",NaN
2,13.4 + 10.3 = ___,NaN
3,Nyzir drinks 1.84 liters of Gatorade. Katie dr...,NaN
4,Tyrek ran a total of 74 minutes in a 2-day per...,NaN
...,...,...
63,Sam does 33 sit-ups 10 times per day. What is ...,NaN
64,Use the standard algorithm to find 2 X 6.,NaN
65,What is the product of 52 and 61,NaN
66,Reese poured 3 ounces of Gatorade into each of...,NaN


In [34]:
from datasets import load_dataset

dataset = pd.read_csv('text.csv')

In [36]:
from datasets import load_dataset

dataset = load_dataset("text.csv")
dataset

FileNotFoundError: Couldn't find a dataset script at /solmathdashboard/app/GTP-2 Model/text.csv/text.csv.py or any data file in the same directory. Couldn't find 'text.csv' on the Hugging Face Hub either: FileNotFoundError: Dataset 'text.csv' doesn't exist on the Hub. If the repo is private, make sure you are authenticated with `use_auth_token=True` after logging in with `huggingface-cli login`.

In [35]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

AttributeError: 'DataFrame' object has no attribute 'map'

In [16]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading: 100%|██████████| 436M/436M [00:44<00:00, 9.77MB/s] 
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceC

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="GTP-2 Model")

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [1]:
trainer.train()

NameError: name 'trainer' is not defined